In [1]:
import json
import time
import func
from multiprocessing import Pool, Manager, Process

import pandas as pd
import os

## データ準備

今回は [郵便番号データの説明 - 日本郵便](https://www.post.japanpost.jp/zipcode/dl/readme.html) からデータを借りてきて検証の材料とする

In [2]:
postal_code_csv_path = os.path.abspath('KEN_ALL.CSV')
postal_code_df = pd.read_csv(postal_code_csv_path, header=None, encoding='cp932', dtype={1: str, 2: str},
                             index_col=None,
                             names=('全国地方公共団体コード（JIS X0401、X0402）', '（旧）郵便番号（5桁）', '郵便番号（7桁）',
                                    '都道府県名（半角カタカナ）', '市区町村名（半角カタカナ）', '町域名（半角カタカナ）',
                                    '都道府県名（漢字）', '市区町村名（漢字）', '町域名（漢字）',
                                    '一町域が二以上の郵便番号で表される場合の表示', '小字毎に番地が起番されている町域の表示',
                                    '丁目を有する町域の場合の表示', '一つの郵便番号で二以上の町域を表す場合の表示',
                                    '更新の表示', '変更理由'))
postal_code_df

,全国地方公共団体コード（JIS X0401、X0402）,（旧）郵便番号（5桁）,郵便番号（7桁）,都道府県名（半角カタカナ）,市区町村名（半角カタカナ）,町域名（半角カタカナ）,都道府県名（漢字）,市区町村名（漢字）,町域名（漢字）,一町域が二以上の郵便番号で表される場合の表示,小字毎に番地が起番されている町域の表示,丁目を有する町域の場合の表示,一つの郵便番号で二以上の町域を表す場合の表示,更新の表示,変更理由
0,1101,060,0600000,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｲｶﾆｹｲｻｲｶﾞﾅｲﾊﾞｱｲ,北海道,札幌市中央区,以下に掲載がない場合,0,0,0,0,0,0
1,1101,064,0640941,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｱｻﾋｶﾞｵｶ,北海道,札幌市中央区,旭ケ丘,0,0,1,0,0,0
2,1101,060,0600041,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾋｶﾞｼ,北海道,札幌市中央区,大通東,0,0,1,0,0,0
3,1101,060,0600042,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(1-19ﾁｮｳﾒ),北海道,札幌市中央区,大通西（１～１９丁目）,1,0,1,0,0,0
4,1101,064,0640820,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(20-28ﾁｮｳﾒ),北海道,札幌市中央区,大通西（２０～２８丁目）,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124551,47381,90714,9071433,ｵｷﾅﾜｹﾝ,ﾔｴﾔﾏｸﾞﾝﾀｹﾄﾐﾁｮｳ,ﾊｲﾐﾅｶ,沖縄県,八重山郡竹富町,南風見仲,0,0,0,0,0,0
124552,47381,90717,9071751,ｵｷﾅﾜｹﾝ,ﾔｴﾔﾏｸﾞﾝﾀｹﾄﾐﾁｮｳ,ﾊﾃﾙﾏ,沖縄県,八重山郡竹富町,波照間,0,0,0,0,0,0
124553,47381,90715,9071544,ｵｷﾅﾜｹﾝ,ﾔｴﾔﾏｸﾞﾝﾀｹﾄﾐﾁｮｳ,ﾊﾄﾏ,沖縄県,八重山郡竹富町,鳩間,0,0,0,0,0,0
124554,47382,90718,9071800,ｵｷﾅﾜｹﾝ,ﾔｴﾔﾏｸﾞﾝﾖﾅｸﾞﾆﾁｮｳ,ｲｶﾆｹｲｻｲｶﾞﾅｲﾊﾞｱｲ,沖縄県,八重山郡与那国町,以下に掲載がない場合,0,0,0,0,0,0


In [3]:
postal_code_df.iloc[:10, :]

,全国地方公共団体コード（JIS X0401、X0402）,（旧）郵便番号（5桁）,郵便番号（7桁）,都道府県名（半角カタカナ）,市区町村名（半角カタカナ）,町域名（半角カタカナ）,都道府県名（漢字）,市区町村名（漢字）,町域名（漢字）,一町域が二以上の郵便番号で表される場合の表示,小字毎に番地が起番されている町域の表示,丁目を有する町域の場合の表示,一つの郵便番号で二以上の町域を表す場合の表示,更新の表示,変更理由
0,1101,060,0600000,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｲｶﾆｹｲｻｲｶﾞﾅｲﾊﾞｱｲ,北海道,札幌市中央区,以下に掲載がない場合,0,0,0,0,0,0
1,1101,064,0640941,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｱｻﾋｶﾞｵｶ,北海道,札幌市中央区,旭ケ丘,0,0,1,0,0,0
2,1101,060,0600041,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾋｶﾞｼ,北海道,札幌市中央区,大通東,0,0,1,0,0,0
3,1101,060,0600042,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(1-19ﾁｮｳﾒ),北海道,札幌市中央区,大通西（１～１９丁目）,1,0,1,0,0,0
4,1101,064,0640820,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(20-28ﾁｮｳﾒ),北海道,札幌市中央区,大通西（２０～２８丁目）,1,0,1,0,0,0
5,1101,060,0600031,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｷﾀ1ｼﾞｮｳﾋｶﾞｼ,北海道,札幌市中央区,北一条東,0,0,1,0,0,0
6,1101,060,0600001,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｷﾀ1ｼﾞｮｳﾆｼ(1-19ﾁｮｳﾒ),北海道,札幌市中央区,北一条西（１～１９丁目）,1,0,1,0,0,0
7,1101,064,0640821,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｷﾀ1ｼﾞｮｳﾆｼ(20-28ﾁｮｳﾒ),北海道,札幌市中央区,北一条西（２０～２８丁目）,1,0,1,0,0,0
8,1101,060,0600032,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｷﾀ2ｼﾞｮｳﾋｶﾞｼ,北海道,札幌市中央区,北二条東,0,0,1,0,0,0
9,1101,060,0600002,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,ｷﾀ2ｼﾞｮｳﾆｼ(1-19ﾁｮｳﾒ),北海道,札幌市中央区,北二条西（１～１９丁目）,1,0,1,0,0,0


## プロセスプールを使った例

[https://docs.python.org/ja/3/library/multiprocessing.html#module-multiprocessing.pool](https://docs.python.org/ja/3/library/multiprocessing.html#module-multiprocessing.pool)

成功時／失敗時に呼ばれるコールバック関数を定義する。
関数は以下の単一の引数を取る。

- 成功時
  - プロセスの処理結果
- 失敗時
  - 例外オブジェクト

In [4]:
success_callback_args_list = []
def success_callback(*args):
  print('pid:', os.getpid())
  print("callback {}".format(args))
  success_callback_args_list.append(os.getpid())

In [5]:
error_callback_args_list = []
def error_callback(*args):
    print("error callback args={}".format(args))
    error_callback_args_list(args)

プロセスプールを作成する。

In [7]:
process_pool = Pool(5, maxtasksperchild=1)
print(process_pool)

<multiprocessing.pool.Pool state=RUN pool_size=5>


apply_async を呼び出して、処理を並行して実行する。

In [8]:
process_list = []
success_callback_args_list = []
for row in postal_code_df.iloc[:10, :].itertuples():
  plain_test = ''.join(row[7:10])
  print(plain_test)
  process_list.append(process_pool.apply_async(func.hash_text, args=(plain_test,) ,callback=success_callback, error_callback=error_callback))

北海道札幌市中央区以下に掲載がない場合
北海道札幌市中央区旭ケ丘
北海道札幌市中央区大通東
北海道札幌市中央区大通西（１～１９丁目）
北海道札幌市中央区大通西（２０～２８丁目）
北海道札幌市中央区北一条東
北海道札幌市中央区北一条西（１～１９丁目）
北海道札幌市中央区北一条西（２０～２８丁目）
北海道札幌市中央区北二条東
北海道札幌市中央区北二条西（１～１９丁目）


In [10]:
print([r.successful() for r in process_list[:]])

[True, True, True, True, True, True, True, True, True, True]


In [11]:
print([r.get()[0] for r in process_list[:]])
print([r.get() for r in process_list[:]])

[25532, 25172, 31200, 25132, 13288, 15640, 5100, 13212, 4456, 22276]
[(25532, '北海道札幌市中央区以下に掲載がない場合', '$argon2id$v=19$m=65536,t=3,p=4$nbQTSWHJPGdZQtS3qsg2tw$EkPI9NEUVNX0bucz2en9qvWUN/17mLFYO23Y3E+Zopw'), (25172, '北海道札幌市中央区旭ケ丘', '$argon2id$v=19$m=65536,t=3,p=4$istzU+AjqGAblPMyv3dFeg$ChNCqitJGlUk8tHqcpz1k+OvYRTX+QfmzflPptKqDwk'), (31200, '北海道札幌市中央区大通東', '$argon2id$v=19$m=65536,t=3,p=4$5tEJpW7ohufx7iUbHD2cMg$h9ELjWNMppsLAWZNbUZg9vEloFVpfUvnepmudo/lGx8'), (25132, '北海道札幌市中央区大通西（１～１９丁目）', '$argon2id$v=19$m=65536,t=3,p=4$pTTWFhgDJvBZmJvqCdXewA$d0IUH0ERpRKq9R5qiPSslJQvJQPL3gnGaYw5NHiXcP8'), (13288, '北海道札幌市中央区大通西（２０～２８丁目）', '$argon2id$v=19$m=65536,t=3,p=4$TdvQIHh7Fic0bjXOeOWZ0Q$gxCe194sN2CN+sgIBWlHYmTu0fCTtWEd5cKDL8OLil0'), (15640, '北海道札幌市中央区北一条東', '$argon2id$v=19$m=65536,t=3,p=4$LZI/cndDje2iTrbll+Hijw$yq+8nkfihsCMdu8tKZuJkN/Qvvp64vtOfmeWZI3HvmA'), (5100, '北海道札幌市中央区北一条西（１～１９丁目）', '$argon2id$v=19$m=65536,t=3,p=4$4+FlhgUD7lM4cSNZnlH4zw$ljjscqr4B5x25StWtixJPZmc761zsULCPDiRre0iuN8'), (13212, '北海道札幌市

In [45]:
success_callback_args_list

[]

In [187]:
process_pool.terminate()

In [18]:
process_list = []
success_callback_args_list = []
sync_manager = Manager()
print(sync_manager)
shared_list = sync_manager.list()
print(shared_list)
process_pool = Pool(5, maxtasksperchild=1)
print(process_pool)

[]
<multiprocessing.pool.Pool state=RUN pool_size=5>


In [19]:
processing_df_list = []
for start_index in range(0, len(postal_code_df), 10000):
  processing_df_list.append(postal_code_df[start_index:start_index+10000])
print([len(df) for df in processing_df_list])

[10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 4556]


In [20]:
for df in processing_df_list:
  result = process_pool.apply_async(
    func.hash_text_df,
    args=(df,),
    kwds={'shared_list': shared_list},
    callback=success_callback,
    error_callback=error_callback)
  process_list.append(result)

In [39]:
print([r.ready() for r in process_list[:]])

[False, False, False, False, False, False, False, False, False, False, False, False, False]


In [46]:
success_callback_args_list

[]

In [43]:
print(len(shared_list[:]))
shared_list[:]

17481


['青森県上北郡野辺地町米内沢',
 '北海道札幌市中央区以下に掲載がない場合',
 '山形県最上郡大蔵村清水',
 '群馬県富岡市内匠',
 '東京都豊島区東池袋サンシャイン６０（３７階）',
 '北海道札幌市中央区旭ケ丘',
 '青森県上北郡七戸町以下に掲載がない場合',
 '群馬県富岡市田篠',
 '山形県最上郡大蔵村南山（４３０番地以上「１７７０－１～２、１８６２－４２、',
 '東京都豊島区東池袋サンシャイン６０（３８階）',
 '北海道札幌市中央区大通東',
 '青森県上北郡七戸町荒熊内',
 '群馬県富岡市田島',
 '山形県最上郡大蔵村１９２３－５を除く」、大谷地、折渡、鍵金野、金山、滝ノ沢、豊牧、沼の台、',
 '東京都豊島区東池袋サンシャイン６０（３９階）',
 '北海道札幌市中央区大通西（１～１９丁目）',
 '青森県上北郡七戸町荒屋',
 '群馬県富岡市富岡',
 '山形県最上郡大蔵村肘折、平林）',
 '東京都豊島区東池袋サンシャイン６０（４０階）',
 '北海道札幌市中央区大通西（２０～２８丁目）',
 '青森県上北郡七戸町有田沢',
 '群馬県富岡市中沢',
 '山形県最上郡大蔵村南山（その他）',
 '東京都豊島区東池袋サンシャイン６０（４１階）',
 '北海道札幌市中央区北一条東',
 '青森県上北郡七戸町家ノ裏',
 '群馬県富岡市中高瀬',
 '山形県最上郡鮭川村以下に掲載がない場合',
 '東京都豊島区東池袋サンシャイン６０（４２階）',
 '北海道札幌市中央区北一条西（１～１９丁目）',
 '青森県上北郡七戸町家ノ上',
 '群馬県富岡市七日市',
 '山形県最上郡鮭川村石名坂',
 '東京都豊島区東池袋サンシャイン６０（４３階）',
 '北海道札幌市中央区北一条西（２０～２８丁目）',
 '青森県上北郡七戸町家ノ下',
 '群馬県富岡市南蛇井',
 '東京都豊島区東池袋サンシャイン６０（４４階）',
 '山形県最上郡鮭川村川口',
 '北海道札幌市中央区北二条東',
 '青森県上北郡七戸町家ノ前',
 '群馬県富岡市野上',
 '東京都豊島区東池袋サンシャイン６０（４５階）',
 '山形県最上郡鮭川村京塚',
 '北海道札幌市中央区北二条西（１～１９丁目）',
 '青森県上北郡七戸町市ノ渡',
 '群馬県富岡市原',
 '